# IIC-3800 Tópicos en CC - NLP UC



In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import transformers

print(transformers.__version__)

4.25.1


In [3]:
from transformers.utils import send_example_telemetry

send_example_telemetry("text_classification_notebook", framework="pytorch")

! pip install evaluate sacrebleu

In [5]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

Found cached dataset opus_books (C:/Users/marce/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
books = books["train"].train_test_split(test_size=0.2)

In [7]:
books["train"][0]

{'id': '53935',
 'translation': {'en': 'She sprang up and said to him—',
  'fr': 'Elle se leva d’un bond et lui dit:'}}

In [8]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [10]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [12]:
import evaluate

metric = evaluate.load("sacrebleu")

In [13]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [14]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation, id. If translation, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
C:\Users\marce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  war

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.825000,1.607834,5.551000,17.614400
2,1.817300,1.588292,5.669800,17.608500


Saving model checkpoint to my_opus_books_model\checkpoint-500
Configuration saved in my_opus_books_model\checkpoint-500\config.json
Model weights saved in my_opus_books_model\checkpoint-500\pytorch_model.bin
tokenizer config file saved in my_opus_books_model\checkpoint-500\tokenizer_config.json
Special tokens file saved in my_opus_books_model\checkpoint-500\special_tokens_map.json
Copy vocab file to my_opus_books_model\checkpoint-500\spiece.model
Saving model checkpoint to my_opus_books_model\checkpoint-1000
Configuration saved in my_opus_books_model\checkpoint-1000\config.json
Model weights saved in my_opus_books_model\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in my_opus_books_model\checkpoint-1000\tokenizer_config.json
Special tokens file saved in my_opus_books_model\checkpoint-1000\special_tokens_map.json
Copy vocab file to my_opus_books_model\checkpoint-1000\spiece.model
Saving model checkpoint to my_opus_books_model\checkpoint-1500
Configuration saved in my_opu

Deleting older checkpoint [my_opus_books_model\checkpoint-6000] due to args.save_total_limit
Saving model checkpoint to my_opus_books_model\checkpoint-8000
Configuration saved in my_opus_books_model\checkpoint-8000\config.json
Model weights saved in my_opus_books_model\checkpoint-8000\pytorch_model.bin
tokenizer config file saved in my_opus_books_model\checkpoint-8000\tokenizer_config.json
Special tokens file saved in my_opus_books_model\checkpoint-8000\special_tokens_map.json
Copy vocab file to my_opus_books_model\checkpoint-8000\spiece.model
Deleting older checkpoint [my_opus_books_model\checkpoint-6500] due to args.save_total_limit
Saving model checkpoint to my_opus_books_model\checkpoint-8500
Configuration saved in my_opus_books_model\checkpoint-8500\config.json
Model weights saved in my_opus_books_model\checkpoint-8500\pytorch_model.bin
tokenizer config file saved in my_opus_books_model\checkpoint-8500\tokenizer_config.json
Special tokens file saved in my_opus_books_model\checkpoi

TrainOutput(global_step=12710, training_loss=1.8360521798242648, metrics={'train_runtime': 2160.7172, 'train_samples_per_second': 94.106, 'train_steps_per_second': 5.882, 'total_flos': 4992588821692416.0, 'train_loss': 1.8360521798242648, 'epoch': 2.0})

In [17]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [25]:
trainer.save_model("my_opus_books_model.h5")

Saving model checkpoint to my_opus_books_model.h5
Configuration saved in my_opus_books_model.h5\config.json
Model weights saved in my_opus_books_model.h5\pytorch_model.bin
tokenizer config file saved in my_opus_books_model.h5\tokenizer_config.json
Special tokens file saved in my_opus_books_model.h5\special_tokens_map.json
Copy vocab file to my_opus_books_model.h5\spiece.model


In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_opus_books_model.h5")
inputs = tokenizer(text, return_tensors="pt").input_ids

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [29]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_opus_books_model.h5")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

loading configuration file my_opus_books_model.h5\config.json
Model config T5Config {
  "_name_or_path": "my_opus_books_model.h5",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {


In [30]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

"Les lègumes partagent les ressources avec des bactéries fixes à l'azote."